In [1]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    LeafSystem_,
    MultibodyPlantConfig,
    Parser,
    Simulator_,
)

def autodiff_to_value(v):
    """Similar to ExtractValue, but retains original shape."""
    shape = v.shape
    return ExtractValue(v).reshape(shape)


def autodiff_to_value_and_grad(v):
    """
    Extracts both value and gradient from AutoDiffXd array `v`.
    """
    value = autodiff_to_value(v)
    grad = ExtractGradient(v)
    return value, grad

def np_print_more_like_matlab():
    np.set_printoptions(
        formatter={"float_kind": lambda x: f"{x: 06.3f}"},
        linewidth=150,
    )
    
np_print_more_like_matlab()

In [2]:
system = LeafSystem_[AutoDiffXd]()
simulator = Simulator_[AutoDiffXd](system)
simulator.AdvanceTo(1.0)

In [3]:
def make_sim_diagram():
    builder = DiagramBuilder()
    config = MultibodyPlantConfig(
        # Option 1: Continuous. Can get gradient.
        time_step=0.0,
        
        # # Option 2: TAMSI. Can get gradient.
        # # Not sure on accuracy, though (regardless of gradient).
        # time_step=0.01,
        # discrete_contact_solver="tamsi",
        
        # # Option 3: SAP. Cannot get gradient when using SceneGraph.
        # # https://github.com/RobotLocomotion/drake/issues/17647
        # time_step=0.01,
        # discrete_contact_solver="sap",
    )
    plant, scene_graph = AddMultibodyPlant(config, builder)
    parser = Parser(plant)
    parser.AddModelsFromUrl(
        "package://drake/manipulation/models/franka_description/urdf/panda_arm.urdf"
    )
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("panda_link0"),
    )
    plant.Finalize()

    frame_G = plant.GetFrameByName("panda_link8")
    # Blech.
    model = plant.GetModelInstanceByName("panda")
    builder.ExportOutput(plant.get_state_output_port(), "state")
    builder.ExportInput(plant.get_actuation_input_port(model), "torque")
    diagram = builder.Build()
    return diagram

def run_sim(diagram, t_final, u, *, T=AutoDiffXd):
    simulator = Simulator_[T](diagram)
    context = simulator.get_context()
    u_port = diagram.GetInputPort("torque")
    x_port = diagram.GetOutputPort("state")
    u_port.FixValue(context, u)
    simulator.AdvanceTo(t_final)
    x = x_port.Eval(context)
    return x

In [4]:
diagram = make_sim_diagram()
diagram_ad = diagram.ToAutoDiffXd()

u = np.ones(7) * 2
u_ad = InitializeAutoDiff(u)

x_ad = run_sim(diagram_ad, t_final=1.0, u=u_ad, T=AutoDiffXd)

x, Jx = autodiff_to_value_and_grad(x_ad)

print(x)
print("---")
print(Jx)

[ 0.278  2.503  1.177 -0.074  5.154  2.572  4.929 -3.489  1.160  4.509 -8.891  7.486  4.838  9.869]
---
[[ 0.382 -0.194 -0.151 -0.091 -0.033 -0.089  0.004]
 [ 0.023  0.006  0.233 -0.068 -0.010  0.024  0.000]
 [ 0.174  0.103  0.532  0.067 -0.083  0.129 -0.007]
 [ 0.098 -0.577  0.701 -0.675 -0.048 -0.255  0.022]
 [ 0.025 -0.008  0.107  0.012  1.814 -0.197 -0.020]
 [ 0.025  0.048  0.093  0.010 -0.117  1.993  0.017]
 [ 0.004 -0.000 -0.011  0.005 -0.021 -0.000  2.461]
 [ 1.072 -1.255 -0.218 -1.307  1.112 -0.331  0.132]
 [ 0.686  1.291 -1.338  3.065 -0.945  0.934 -0.129]
 [ 0.952  1.895 -1.265  4.784 -1.510  1.702 -0.204]
 [ 2.917 -3.894  12.757 -11.432  0.747 -3.282  0.311]
 [-0.026 -0.070 -1.659  0.905  3.763 -3.012 -0.035]
 [ 0.125 -0.669  0.748 -0.994 -0.565  1.246  0.104]
 [-0.013  0.036 -0.178  0.135  0.001 -0.301  4.882]]
